# Build the Neural Network

Neural networks comprise of layers/modules that perform operations on data. The `torch.nn` namespace provides all the building blocks needed to build our own neural network. Every module in PyTorch subclasses the `nn.Module`. A neural network is a module itself that consists of other modules (layers). This nested structure allows for building and managing complext architectures easily. Here we will build a neural network to classify images in the FashionMNIST dataset.

In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [2]:
# Get device for training: if the accelerator such as (CUDA, MPS, MTIA, XPU) is available we will use it, 
# otherwise we use the CPU.
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


## Define the class
We define our neural network by subclassing `nn.Module`, and initialize the neural network layers in `__init__`. Every `nn.Module` subclass implements the operations on input data in the `forward` method.

In [3]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )
    
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [4]:
# We can create an instance of NeuralNetwork, and move it to the device, and print its structure

model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


To use the model, we pass the input data to the NeuralNetwork object directly. This executes the model's `forward`, along with some background operations, we don't call `model.forward()` directly!

Calling the model on the input returns a 2-dimensional tensor with dim=0 corresponding to each output of 10 raw predicted values for each class, and dim=1 corresponding to the individual values of each output. We get the prediction probabilities by passing it through an instance of the nn.Softmax module.
*(Translation: The model outputs a 2-dimensional tensor. dim=0, which you can consider as rows, contains output for different inputs. So if you have 8 images as input, the model generates corresponding predictions for each of those 8 images. dim=1, which represents columns, contains the raw predicted values for each class. Since there are 10 classes, each image will have 10 values. These values are then passed through an instance of nn.Softmax, converting them into prediction probabilities)* 

In [6]:
X = torch.rand(1, 28, 28, device=device) # random image created of 28 * 28 value
logits = model(X)
print(logits)
pred_probab = nn.Softmax(dim=1)(logits)
print(pred_probab)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

tensor([[-0.0378, -0.0070,  0.0231,  0.0181, -0.0485, -0.0481,  0.0624,  0.0101,
         -0.0637,  0.0716]], device='cuda:0', grad_fn=<AddmmBackward0>)
tensor([[0.0964, 0.0994, 0.1024, 0.1019, 0.0954, 0.0954, 0.1065, 0.1011, 0.0939,
         0.1075]], device='cuda:0', grad_fn=<SoftmaxBackward0>)
Predicted class: tensor([9], device='cuda:0')


## Model Layers

We will manually pass some random image values through the created neural network model and see how it will work.

Let’s break down the layers in the FashionMNIST model. To illustrate it, we will take a sample minibatch of 3 images of size 28x28 and see what happens to it as we pass it through the network.

In [7]:
input_image = torch.rand(3, 28, 28)
print(input_image.size())

torch.Size([3, 28, 28])


### nn.Flatten
We initialize the `nn.Flatten` layer to convert each 2D 28*28 image into a contiguous array of 784 pixel values (the minibatch dimension (at dim=0) is maintained)

In [8]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


### nn.Linear
The linear layer is a module that applies a linear transformation on the input using its stored weights and biases.

In [9]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


### nn.ReLU
Non-linear activations are what create the complex mappings between the model's inputs and outputs. They are applied after linear transformations to introduce *nonlinearity*, helping neural networks learn a wide variety of phenomena. In this model, we are using `nn.ReLU` between our linear layers, but there's other activations to introduce non-linearity in your model.

In [10]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[-2.9018e-01,  1.4587e-02,  9.4504e-01, -2.3928e-01, -2.2437e-01,
          1.3890e-02,  2.0329e-01, -3.5941e-01, -4.7915e-01, -8.9112e-02,
          4.0512e-01,  2.0875e-01,  1.3018e-01,  6.4789e-01, -4.2096e-01,
         -3.7672e-01,  2.2942e-01,  2.8721e-01, -2.4472e-01,  8.3244e-01],
        [-3.5877e-01,  1.8331e-01,  8.6834e-01,  7.7511e-02, -2.4233e-01,
          2.3676e-01, -9.1214e-02, -4.0245e-01, -5.3652e-02, -4.0773e-02,
          3.3369e-01, -1.8694e-01, -2.0394e-01,  4.3379e-01,  9.8646e-02,
         -3.6620e-02, -1.1547e-01,  2.3175e-01,  4.4620e-02,  5.0796e-01],
        [-3.6350e-01,  1.3775e-01,  9.7806e-01,  1.9604e-02,  1.6640e-01,
          9.1765e-02,  4.6270e-02, -2.0042e-01, -2.2890e-01,  2.1084e-01,
          5.0171e-01,  1.3631e-02, -1.3999e-01,  5.5846e-01, -2.1052e-01,
         -5.1434e-04,  3.3337e-01,  2.3563e-02, -2.1060e-02,  5.3515e-01]],
       grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0000, 0.0146, 0.9450, 0.0000, 0.0000,

### nn.Sequential
`nn.Sequential` is an ordered container of modules. The data is passed through all the modules in the same order as defined. You can use sequential containers to put together a quick network like *seq_module*. 

In [12]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20,10)
)
input_image = torch.rand(3, 28, 28)
logits = seq_modules(input_image)
print(logits)

tensor([[-0.1132, -0.4159, -0.1520,  0.2325,  0.1479, -0.1300,  0.1172,  0.0888,
          0.2399,  0.1993],
        [-0.2145, -0.5023, -0.1036,  0.2922, -0.0317, -0.1754,  0.1467,  0.0078,
          0.1521,  0.1922],
        [-0.1821, -0.4848, -0.0367,  0.1924,  0.0858, -0.0822,  0.0804,  0.0683,
          0.3049,  0.2761]], grad_fn=<AddmmBackward0>)


### nn.Softmax
The last linear layer of the neural network returns *logits* - raw values in [-infty, infty] - which are passed to the `nn.Softmax` module. The logits are scaled to values [0, 1] representing the model's predicted probabilities for each class. **dim** parameter indicates the dimesion along which the values must sum to 1.

In [14]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)
print(pred_probab)

tensor([[0.0857, 0.0633, 0.0824, 0.1211, 0.1113, 0.0843, 0.1079, 0.1049, 0.1220,
         0.1171],
        [0.0807, 0.0605, 0.0901, 0.1339, 0.0969, 0.0839, 0.1158, 0.1007, 0.1164,
         0.1212],
        [0.0797, 0.0588, 0.0921, 0.1158, 0.1041, 0.0880, 0.1036, 0.1023, 0.1296,
         0.1259]], grad_fn=<SoftmaxBackward0>)


## Model Parameters
Many layers inside a neural network are *parameterized*, i.e., have associated weights and biases that are optimized during training. Subclassing `nn.Module` automatically tracks all fields defined inside your model object, and makes all parameters accessible using our model's *parameters()* or *named_parameters()* methods.

Below we will iterate over each parameter, and print its size and a preview of its values.

In [15]:
print(f"Model Structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values: {param[:2]} \n")

Model Structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values: tensor([[ 0.0342, -0.0212,  0.0278,  ...,  0.0198, -0.0158, -0.0201],
        [-0.0220,  0.0036,  0.0032,  ...,  0.0294,  0.0310,  0.0091]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values: tensor([-0.0276,  0.0329], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values: tensor([[-0.0301, -0.0210,  0.0049,  ...,  0.0387,  0.0208,  0.0040],
        [-0.0096, -0.0310, -0.0145,  ..., -0.0198, -0.0064,  0.0364]],
       device='cuda:0', grad_fn=<Slice